# CAGR Analysis: Top 5 Four-Wheeler EV Makers (2022-2024)

This notebook analyzes the Compounded Annual Growth Rate (CAGR) for the top 5 four-wheeler electric vehicle manufacturers in India from 2022 to 2024. We'll calculate the CAGR, analyze sales trends, examine market share evolution, and create visualizations to understand the competitive landscape.

## 1. Import Required Libraries

First, let's import all the necessary libraries for our analysis:

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display
import os
from pathlib import Path

# Set plotting styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')

# Increase font sizes for better readability
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12

## 2. Load and Prepare Data

Now, let's load the EV sales data from the CSV file and prepare it for analysis:

In [2]:
# Path to the data files
DATA_DIR = Path(os.path.dirname(os.getcwd())) / "data"
EV_SALES_BY_MAKERS_PATH = DATA_DIR / "processed" / "ev_sales_by_makers_cleaned_20250806.csv"

# Load the data
df = pd.read_csv(EV_SALES_BY_MAKERS_PATH)

# Display basic information about the dataset
print("Dataset Info:")
print("-" * 50)
print(df.info())
print("\nFirst few rows:")
print("-" * 50)
display(df.head())

Dataset Info:
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    816 non-null    object
 1   vehicle_category        816 non-null    object
 2   maker                   816 non-null    object
 3   electric_vehicles_sold  816 non-null    int64 
 4   uuid                    816 non-null    object
 5   year                    816 non-null    int64 
 6   month                   816 non-null    object
 7   cumulative_sales        816 non-null    int64 
 8   segment                 816 non-null    object
dtypes: int64(3), object(6)
memory usage: 57.5+ KB
None

First few rows:
--------------------------------------------------


,date,vehicle_category,maker,electric_vehicles_sold,uuid,year,month,cumulative_sales,segment
0,2021-04-01,2-Wheelers,OLA ELECTRIC,0,465931d3-9bd6-43b5-ab1f-aace65b11dfb,2021,April,0,Two-Wheeler Manufacturer
1,2022-04-01,2-Wheelers,OKAYA EV,0,1d7ca2e2-988c-42e9-9d44-8b08a759fbbf,2022,April,0,Two-Wheeler Manufacturer
2,2021-05-01,2-Wheelers,OLA ELECTRIC,0,9da18de8-1173-4717-a894-f3b0748e3249,2021,May,0,Two-Wheeler Manufacturer
3,2021-06-01,2-Wheelers,OLA ELECTRIC,0,ac373721-db45-46df-bc6a-86f0d663253a,2021,June,0,Two-Wheeler Manufacturer
4,2021-07-01,2-Wheelers,OLA ELECTRIC,0,9919575b-4d9d-4823-a9a1-f2a1b7c037c7,2021,July,0,Two-Wheeler Manufacturer


In [3]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Create fiscal year column - in India fiscal year starts in April
df['fiscal_year'] = df['date'].dt.year + np.where(df['date'].dt.month >= 4, 1, 0)

# Check unique fiscal years to confirm our calculation
print("Available fiscal years:")
print(sorted(df['fiscal_year'].unique()))

# Check unique vehicle segments to confirm we have Four-Wheeler data
print("\nAvailable vehicle segments:")
print(df['segment'].unique())

Available fiscal years:
[np.int64(2022), np.int64(2023), np.int64(2024)]

Available vehicle segments:
['Two-Wheeler Manufacturer' 'Four-Wheeler Manufacturer']


## 3. Calculate Total Sales by Maker

Let's filter the data for 4-wheeler manufacturers and calculate the total sales for each maker:

In [4]:
# Filter for only 4-wheeler manufacturers
df_4w = df[df['segment'] == 'Four-Wheeler Manufacturer']

# Check the resulting data
print("Filtered 4-wheeler data:")
print(f"Number of records: {len(df_4w)}")
print(f"Number of unique makers: {df_4w['maker'].nunique()}")

# Display first few rows of filtered data
display(df_4w.head())

Filtered 4-wheeler data:
Number of records: 360
Number of unique makers: 10


,date,vehicle_category,maker,electric_vehicles_sold,uuid,year,month,cumulative_sales,segment,fiscal_year
9,2021-04-01,4-Wheelers,BYD India,0,c57fec52-2208-46e0-933c-7bd100265253,2021,April,0,Four-Wheeler Manufacturer,2022
10,2021-04-01,4-Wheelers,PCA Automobiles,0,058777d2-ee66-4d2a-b050-e95bfdf75870,2021,April,0,Four-Wheeler Manufacturer,2022
11,2021-04-01,4-Wheelers,BMW India,0,4cdb0c49-c11c-40f3-b7af-f070dd2bfc4f,2021,April,0,Four-Wheeler Manufacturer,2022
12,2021-04-01,4-Wheelers,Volvo Auto India,0,9e338527-5dab-4026-9238-69477647121b,2021,April,0,Four-Wheeler Manufacturer,2022
13,2021-04-01,4-Wheelers,KIA Motors,0,6a0c21ff-98f4-4bf2-b44c-178b156ed89c,2021,April,0,Four-Wheeler Manufacturer,2022


In [5]:
# Group by maker and fiscal year, then sum EV sales
sales_by_maker_year = df_4w.groupby(['maker', 'fiscal_year'])['electric_vehicles_sold'].sum().reset_index()

# Calculate total sales across all years for each maker
total_sales_by_maker = sales_by_maker_year.groupby('maker')['electric_vehicles_sold'].sum().reset_index()
total_sales_by_maker = total_sales_by_maker.sort_values('electric_vehicles_sold', ascending=False)

# Display total sales by maker
print("Total 4-Wheeler EV Sales by Manufacturer (All Years):")
print("-" * 50)
display(total_sales_by_maker.head(10))

Total 4-Wheeler EV Sales by Manufacturer (All Years):
--------------------------------------------------


,maker,electric_vehicles_sold
8,Tata Motors,88935
5,Mahindra and Mahindra,41193
4,MG Motor,13753
1,BYD India,2419
2,Hyundai Motor,2076
7,PCA Automobiles,1684
0,BMW India,1370
9,Volvo Auto India,568
3,KIA Motors,557
6,Mercedes-Benz AG,388


## 4. Identify Top 5 Makers by Sales Volume

Now, let's identify the top 5 makers by total sales volume and analyze their performance:

In [6]:
# Select top 5 makers by total sales
top_5_makers = total_sales_by_maker.head(5)['maker'].tolist()
print("Top 5 Makers by Total Sales:")
print("-" * 50)
for i, maker in enumerate(top_5_makers, 1):
    print(f"{i}. {maker}")

# Filter sales data for top 5 makers
top_5_sales = sales_by_maker_year[sales_by_maker_year['maker'].isin(top_5_makers)]

# Pivot data to get years as columns
pivot_sales = top_5_sales.pivot(index='maker', columns='fiscal_year', values='electric_vehicles_sold').reset_index()

# Display the pivoted data
print("\nSales by Fiscal Year for Top 5 Makers:")
print("-" * 50)
display(pivot_sales)

Top 5 Makers by Total Sales:
--------------------------------------------------
1. Tata Motors
2. Mahindra and Mahindra
3. MG Motor
4. BYD India
5. Hyundai Motor

Sales by Fiscal Year for Top 5 Makers:
--------------------------------------------------


fiscal_year,maker,2022,2023,2024
0,BYD India,33,920,1466
1,Hyundai Motor,110,576,1390
2,MG Motor,1647,3277,8829
3,Mahindra and Mahindra,4042,13805,23346
4,Tata Motors,12708,28046,48181


## 5. Calculate CAGR for Top Makers

Let's calculate the Compound Annual Growth Rate (CAGR) for each of the top 5 makers from 2022 to 2024:

In [7]:
# Define CAGR calculation function
def calculate_cagr(start_value, end_value, years):
    """Calculate Compound Annual Growth Rate"""
    if start_value <= 0:  # Avoid division by zero
        return None
    return ((end_value / start_value) ** (1/years) - 1) * 100

# Ensure all required years exist in the pivot table
for year in [2022, 2023, 2024]:
    if year not in pivot_sales.columns:
        pivot_sales[year] = np.nan

# Calculate CAGR for each maker (2022 to 2024)
pivot_sales['CAGR (%)'] = pivot_sales.apply(
    lambda row: calculate_cagr(row[2022], row[2024], 2) if pd.notnull(row[2022]) and pd.notnull(row[2024]) else None,
    axis=1
)

# Sort by CAGR
pivot_sales_sorted = pivot_sales.sort_values('CAGR (%)', ascending=False)

# Display results
print("CAGR (%) for Top 5 4-Wheeler EV Makers (2022-2024):")
print("-" * 70)
display(pivot_sales_sorted[['maker', 2022, 2023, 2024, 'CAGR (%)']].rename(
    columns={2022: '2022 Sales', 2023: '2023 Sales', 2024: '2024 Sales'}
))

CAGR (%) for Top 5 4-Wheeler EV Makers (2022-2024):
----------------------------------------------------------------------


fiscal_year,maker,2022 Sales,2023 Sales,2024 Sales,CAGR (%)
0,BYD India,33,920,1466,566.515134
1,Hyundai Motor,110,576,1390,255.476633
3,Mahindra and Mahindra,4042,13805,23346,140.330055
2,MG Motor,1647,3277,8829,131.530899
4,Tata Motors,12708,28046,48181,94.714952


## 6. Create CAGR Visualization

Let's create visualizations to better understand and compare the CAGR of the top 5 manufacturers:

In [8]:
# Create a bar chart for CAGR comparison using Plotly
result_df = pivot_sales_sorted.copy()
result_df = result_df.rename(columns={2022: '2022 Sales', 2023: '2023 Sales', 2024: '2024 Sales'})

fig = go.Figure()

# Add bar chart for CAGR
fig.add_trace(
    go.Bar(
        x=result_df['maker'],
        y=result_df['CAGR (%)'],
        text=result_df['CAGR (%)'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A"),
        textposition='outside',
        marker_color=px.colors.sequential.Blues[-5:],
        marker_line_color='rgb(8,48,107)',
        marker_line_width=1.5,
        name='CAGR (%)'
    )
)

# Add horizontal line at 0% for reference
fig.add_hline(
    y=0, 
    line_dash="dash", 
    line_color="red",
    annotation_text="0% Growth", 
    annotation_position="bottom right"
)

# Add average CAGR reference line
avg_cagr = result_df['CAGR (%)'].mean()
if not pd.isna(avg_cagr):
    fig.add_hline(
        y=avg_cagr, 
        line_dash="dot", 
        line_color="green",
        annotation_text=f"Avg CAGR: {avg_cagr:.2f}%", 
        annotation_position="top right"
    )

# Update layout
fig.update_layout(
    title={
        'text': 'CAGR (%) for Top 5 4-Wheeler EV Makers (2022-2024)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20)
    },
    xaxis_title='EV Maker',
    yaxis_title='CAGR (%)',
    template='plotly_white',
    height=500,
    width=900,
    yaxis=dict(
        ticksuffix='%',
        zeroline=False,
        gridcolor='lightgray'
    ),
    plot_bgcolor='white'
)

fig.show()

## 7. Year-over-Year Sales Analysis

Let's analyze the year-over-year sales trends for each of the top 5 manufacturers:

In [9]:
# Create a line chart to visualize sales growth over time
years = [2022, 2023, 2024]
sales_data = pd.melt(
    result_df, 
    id_vars=['maker', 'CAGR (%)'],
    value_vars=['2022 Sales', '2023 Sales', '2024 Sales'],
    var_name='Year',
    value_name='Sales'
)

# Extract just the year from the Year column
sales_data['Year'] = sales_data['Year'].str.extract(r'(\d+)')

# Create line chart with improved styling
fig = px.line(
    sales_data, 
    x='Year', 
    y='Sales', 
    color='maker',
    markers=True,
    title='4-Wheeler EV Sales Growth by Top Makers (2022-2024)',
    template='plotly_white',
    labels={'Sales': 'Units Sold', 'Year': 'Year'},
    color_discrete_sequence=px.colors.qualitative.Bold
)

# Add data labels
for maker in sales_data['maker'].unique():
    maker_data = sales_data[sales_data['maker'] == maker]
    fig.add_trace(
        go.Scatter(
            x=maker_data['Year'],
            y=maker_data['Sales'],
            mode='text',
            text=maker_data['Sales'].apply(lambda x: f"{int(x):,}" if pd.notnull(x) else "N/A"),
            textposition='top center',
            showlegend=False,
            textfont=dict(color='black')
        )
    )

# Update the layout
fig.update_layout(
    legend_title_text='EV Maker',
    xaxis=dict(
        tickmode='array', 
        tickvals=['2022', '2023', '2024'],
        ticktext=['FY 2022', 'FY 2023', 'FY 2024']
    ),
    height=500,
    width=900,
    yaxis_title='Units Sold',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
)

# Update traces to make lines thicker and markers larger
fig.update_traces(
    selector=dict(mode='lines+markers'),
    line=dict(width=3),
    marker=dict(size=10)
)

fig.show()

## 8. Detailed Manufacturer Performance Analysis

Let's create a comprehensive analysis table showing various performance metrics for each manufacturer:

In [10]:
# Calculate year-over-year growth rates
detailed_df = result_df.copy()

# Calculate YoY growth
detailed_df['Growth 2022-2023 (%)'] = ((detailed_df['2023 Sales'] / detailed_df['2022 Sales']) - 1) * 100
detailed_df['Growth 2023-2024 (%)'] = ((detailed_df['2024 Sales'] / detailed_df['2023 Sales']) - 1) * 100

# Calculate market share for each year
for year in ['2022 Sales', '2023 Sales', '2024 Sales']:
    detailed_df[f'{year} Market Share (%)'] = (detailed_df[year] / detailed_df[year].sum()) * 100

# Calculate market share change
detailed_df['Market Share Change (2022-2024)'] = detailed_df['2024 Sales Market Share (%)'] - detailed_df['2022 Sales Market Share (%)']

# Reorder columns for better presentation
column_order = [
    'maker', 
    '2022 Sales', '2022 Sales Market Share (%)',
    '2023 Sales', '2023 Sales Market Share (%)', 'Growth 2022-2023 (%)',
    '2024 Sales', '2024 Sales Market Share (%)', 'Growth 2023-2024 (%)',
    'Market Share Change (2022-2024)', 'CAGR (%)'
]

# Sort by 2024 market share
detailed_df = detailed_df.sort_values('2024 Sales', ascending=False)[column_order]

# Round decimal places for better presentation
detailed_df = detailed_df.round(2)

# Display the detailed analysis table
print("Detailed Performance Analysis of Top 5 4-Wheeler EV Makers (2022-2024)")
print("-" * 120)
display(detailed_df)

Detailed Performance Analysis of Top 5 4-Wheeler EV Makers (2022-2024)
------------------------------------------------------------------------------------------------------------------------


fiscal_year,maker,2022 Sales,2022 Sales Market Share (%),2023 Sales,2023 Sales Market Share (%),Growth 2022-2023 (%),2024 Sales,2024 Sales Market Share (%),Growth 2023-2024 (%),Market Share Change (2022-2024),CAGR (%)
4,Tata Motors,12708,68.54,28046,60.15,120.70,48181,57.90,71.79,-10.64,94.71
3,Mahindra and Mahindra,4042,21.80,13805,29.61,241.54,23346,28.06,69.11,6.25,140.33
2,MG Motor,1647,8.88,3277,7.03,98.97,8829,10.61,169.42,1.73,131.53
0,BYD India,33,0.18,920,1.97,2687.88,1466,1.76,59.35,1.58,566.52
1,Hyundai Motor,110,0.59,576,1.24,423.64,1390,1.67,141.32,1.08,255.48


## 9. Market Share Evolution Analysis

Let's analyze how the market share distribution has changed over the years for the top 5 manufacturers:

In [11]:
# Create a stacked bar chart showing market share evolution
market_share_data = []
for year_col, year_label in zip(['2022 Sales', '2023 Sales', '2024 Sales'], ['2022', '2023', '2024']):
    for _, row in result_df.iterrows():
        maker = row['maker']
        sales = row[year_col]
        share = (sales / result_df[year_col].sum()) * 100
        market_share_data.append({
            'Year': year_label,
            'Maker': maker,
            'Market Share (%)': share
        })

ms_df = pd.DataFrame(market_share_data)

# Create stacked bar chart
fig = px.bar(
    ms_df,
    x='Year',
    y='Market Share (%)',
    color='Maker',
    title='Market Share Evolution by Manufacturer (2022-2024)',
    template='plotly_white',
    text='Market Share (%)',
    color_discrete_sequence=px.colors.qualitative.Bold
)

# Format text labels
fig.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='inside'
)

# Update layout
fig.update_layout(
    xaxis=dict(
        tickmode='array', 
        tickvals=['2022', '2023', '2024'],
        ticktext=['FY 2022', 'FY 2023', 'FY 2024']
    ),
    yaxis=dict(
        title='Market Share (%)',
        ticksuffix='%'
    ),
    legend_title='Manufacturer',
    height=500,
    width=900
)

fig.show()

Let's also create a combined visualization showing both sales and CAGR:

In [12]:
# Create a combined visualization with sales and CAGR
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Sort data by CAGR
result_sorted = result_df.sort_values('CAGR (%)', ascending=False)

# Add bars for 2022, 2023, 2024 sales
for year, color in zip(['2022 Sales', '2023 Sales', '2024 Sales'], ['#D4E6F1', '#7FB3D5', '#2874A6']):
    fig.add_trace(
        go.Bar(
            x=result_sorted['maker'],
            y=result_sorted[year],
            name=year,
            marker_color=color,
            text=result_sorted[year].apply(lambda x: f"{int(x):,}" if pd.notnull(x) else "N/A"),
            textposition='inside',
            width=0.2,
            offset=(-0.25 + 0.25*list(['2022 Sales', '2023 Sales', '2024 Sales']).index(year)),
            textfont=dict(color='black', size=10),
        ),
        secondary_y=False
    )

# Add line for CAGR
fig.add_trace(
    go.Scatter(
        x=result_sorted['maker'],
        y=result_sorted['CAGR (%)'],
        name='CAGR (%)',
        mode='lines+markers+text',
        marker=dict(size=12, symbol='diamond', color='#E74C3C'),
        line=dict(width=3, color='#E74C3C', dash='dot'),
        text=result_sorted['CAGR (%)'].apply(lambda x: f"{x:.1f}%" if pd.notnull(x) else "N/A"),
        textposition='top center',
        textfont=dict(color='#E74C3C')
    ),
    secondary_y=True
)

# Update layout
fig.update_layout(
    title_text='Sales Growth and CAGR by Top 4-Wheeler EV Makers (2022-2024)',
    barmode='group',
    template='plotly_white',
    legend=dict(
        orientation='h',
        y=1.1,
        x=0.5,
        xanchor='center'
    ),
    height=600,
    width=1000,
    margin=dict(t=120)
)

# Update axes titles
fig.update_yaxes(title_text='Units Sold', secondary_y=False)
fig.update_yaxes(title_text='CAGR (%)', ticksuffix='%', secondary_y=True)
fig.update_xaxes(title_text='EV Maker')

fig.show()

## 10. Build Interactive Dashboard Elements

Now let's prepare elements that would be used in a Streamlit dashboard with filters and KPIs:

In [13]:
# Define a function to calculate CAGR for any two years
def calculate_cagr_for_period(df_4w, start_year, end_year, top_n=5):
    """Calculate CAGR for top N manufacturers between any two years"""
    # Group by maker and fiscal year, sum the EV sales
    sales_by_maker_year = df_4w.groupby(['maker', 'fiscal_year'])['electric_vehicles_sold'].sum().reset_index()
    
    # Calculate total sales across all years to find top N makers
    total_sales_by_maker = sales_by_maker_year.groupby('maker')['electric_vehicles_sold'].sum().reset_index()
    top_makers = total_sales_by_maker.sort_values('electric_vehicles_sold', ascending=False).head(top_n)['maker'].tolist()
    
    # Filter for only the top N makers
    top_sales = sales_by_maker_year[sales_by_maker_year['maker'].isin(top_makers)]
    
    # Pivot data to get years as columns
    pivot_data = top_sales.pivot(index='maker', columns='fiscal_year', values='electric_vehicles_sold').reset_index()
    
    # Calculate CAGR
    years_diff = end_year - start_year
    pivot_data['CAGR (%)'] = pivot_data.apply(
        lambda row: calculate_cagr(row[start_year], row[end_year], years_diff) if pd.notnull(row[start_year]) and pd.notnull(row[end_year]) else None, 
        axis=1
    )
    
    # Sort by CAGR
    pivot_data = pivot_data.sort_values('CAGR (%)', ascending=False)
    
    return pivot_data

In [14]:
# Define Key Performance Indicators (KPIs) calculation functions
def calculate_kpis(result_df, start_year, end_year):
    """Calculate KPIs for the dashboard"""
    kpis = {}
    
    # Find the top performer by CAGR
    if not result_df.empty and 'CAGR (%)' in result_df.columns and not result_df['CAGR (%)'].isna().all():
        top_idx = result_df['CAGR (%)'].idxmax()
        kpis['top_cagr_maker'] = result_df.loc[top_idx, 'maker']
        kpis['top_cagr_value'] = result_df.loc[top_idx, 'CAGR (%)']
    else:
        kpis['top_cagr_maker'] = "N/A"
        kpis['top_cagr_value'] = 0
    
    # Find the manufacturer with highest sales in the most recent year
    end_year_col = f'{end_year} Sales'
    if end_year_col in result_df.columns and not result_df[end_year_col].isna().all():
        top_sales_idx = result_df[end_year_col].idxmax()
        kpis['top_sales_maker'] = result_df.loc[top_sales_idx, 'maker']
        kpis['top_sales_value'] = result_df.loc[top_sales_idx, end_year_col]
    else:
        kpis['top_sales_maker'] = "N/A"
        kpis['top_sales_value'] = 0
    
    # Calculate industry average CAGR
    kpis['avg_cagr'] = result_df['CAGR (%)'].mean() if not result_df['CAGR (%)'].isna().all() else 0
    
    # Calculate total sales growth for the industry
    start_year_col = f'{start_year} Sales'
    if start_year_col in result_df.columns and end_year_col in result_df.columns:
        total_start = result_df[start_year_col].sum()
        total_end = result_df[end_year_col].sum()
        if total_start > 0:
            kpis['industry_growth'] = ((total_end / total_start) ** (1/(end_year-start_year)) - 1) * 100
        else:
            kpis['industry_growth'] = 0
    else:
        kpis['industry_growth'] = 0
    
    return kpis

In [15]:
# Example of dashboard KPIs with our current data
kpis = calculate_kpis(detailed_df, 2022, 2024)

print("Key Performance Indicators for Dashboard:")
print("-" * 50)
print(f"Highest CAGR Manufacturer: {kpis['top_cagr_maker']} with {kpis['top_cagr_value']:.2f}%")
print(f"Top Seller (2024): {kpis['top_sales_maker']} with {int(kpis['top_sales_value']):,} units")
print(f"Industry Average CAGR: {kpis['avg_cagr']:.2f}%")
print(f"Overall Industry Growth (CAGR): {kpis['industry_growth']:.2f}%")

Key Performance Indicators for Dashboard:
--------------------------------------------------
Highest CAGR Manufacturer: BYD India with 566.52%
Top Seller (2024): Tata Motors with 48,181 units
Industry Average CAGR: 237.71%
Overall Industry Growth (CAGR): 111.85%


## Summary of CAGR Analysis

Based on our comprehensive analysis of the top 5 four-wheeler EV manufacturers from 2022 to 2024, we can draw the following conclusions:

1. **CAGR Performance**: The analysis shows significant variations in growth rates among the top manufacturers, with CAGR values ranging from [lowest CAGR] to [highest CAGR].

2. **Market Leadership**: The top performer in terms of CAGR is [top CAGR maker], demonstrating exceptional growth at [top CAGR value]%, which is [comparison] the industry average of [avg CAGR]%.

3. **Sales Volume Leader**: [Top sales maker] leads in absolute sales volume for 2024 with [top sales value] units sold, showing its dominant market position.

4. **Market Share Evolution**: The market share distribution has [increased/decreased] in concentration, with the top manufacturer now holding [current share]% of the market, compared to [past share]% in 2022.

5. **Growth Patterns**: Most manufacturers show [consistent/inconsistent] growth patterns, with [observations about year-over-year trends].

This analysis provides valuable insights into the competitive dynamics of the 4-wheeler EV market in India, highlighting which manufacturers are gaining momentum and which might be losing ground in this rapidly evolving industry.